In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

2024-06-18 16:42:13.608619: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 16:42:14.433427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 16:42:14.433498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 16:42:14.574698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 16:42:14.824387: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# load the trained model
gmp_model = tf.keras.models.load_model('models/trained_gmp_model_dense_32.h5')

2024-06-18 16:42:27.558497: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [3]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [4]:
# This model does not have a Dense layer that can be used to extract features that can be used in a downstream bayesian TS model. So another model was retrained - trained_gmp_model_dense_32.h5

In [5]:
# feature extractor input
feat_ext_input = gmp_model.input

In [6]:
# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

In [7]:
feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [8]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [9]:
# We might need to be cautious here though, as dropout will set some neurons to 0, and the extracted features will therefore become zero - workaround - use the Dense layer to extract features before the pred head (and the dropout layer)

Locate the previously preprocessed data

In [10]:
# We currently have just a single block - 0103

In [11]:
prev_preprocessed_image_loc = "all_preprocessed_data/Block_0103/sub_images_and_counts"

In [12]:
folder_contents = os.listdir(prev_preprocessed_image_loc)
folder_contents.sort()

In [19]:
# may need to write a function that does feature extraction as well as get the respective density values for each subwindow 

In [32]:
def extract_features_and_get_counts(model, preprocessed_path, all_in_folder, digits_image, digits_dense, character, store_path):
    necessary_sub_images = [item for item in all_in_folder if item.split(".")[0][-digits_image:] == ('_' + character)]
    necessary_sub_images.sort()
    necessary_sub_dense_maps = [item for item in all_in_folder if item.split(".")[0][-digits_dense:] == ('_' + character + '_density')]
    necessary_sub_dense_maps.sort()
    print(necessary_sub_images[0])
    # check if the image and density maps tally
    print(np.mean([item.split('.')[0] for item in necessary_sub_images] == [item.split(".")[0][:-8] for item in necessary_sub_dense_maps]))
    
    collect_extracted_features = []
    collect_corresponding_counts = []

    # collect the subwindow-wise features and corresponding counts
    for i in range(len(necessary_sub_images)):
        # load the subimage
        load_image = np.load(os.path.join(preprocessed_path, necessary_sub_images[i]))
        # reshape it to get the prediction
        load_image_reshape = load_image[None, ...]
        # get the prediction from the model
        extracted_features = model.predict(load_image_reshape, verbose = 0)
        collect_extracted_features.append(extracted_features)

        # load the count map
        load_count_map = np.load(os.path.join(preprocessed_path, necessary_sub_dense_maps[i]))
        count = np.sum(load_count_map)
        collect_corresponding_counts.append(count)

    # collect all extracted features data and create a dataframe
    Features_data = pd.DataFrame(np.vstack(collect_extracted_features), columns = ['feature_' + str(i) for i in range(np.vstack(collect_extracted_features).shape[1])])
    # dataframe for counts
    counts_subwindow = pd.DataFrame(collect_corresponding_counts, columns = ['tassel_count'])
    # combine the two dataframes
    combined_df = pd.concat((Features_data,counts_subwindow), axis = 1 )
    # save this dataframe
    save_path_name = store_path + 'extracted_features_sub_window_' + character + '.csv'
    combined_df.to_csv(save_path_name, index = False)

    return(combined_df)

In [33]:
s_path = 'all_preprocessed_data/Block_0103//TS_ready_data_frames/'

In [34]:
# try the function
hello_1 = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, '1', s_path)

Block0103_2020_08_03_1.npy
1.0


In [35]:
hello_1.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,tassel_count
0,0.042926,0.0,0.185817,0.0,0.0,0.164636,0.143964,0.150653,0.167357,0.0,...,0.0,0.0,0.114256,0.204056,0.0,0.189904,0.0,0.0,0.271111,5.008098
1,0.074354,0.0,0.264423,0.0,0.0,0.202246,0.195495,0.203582,0.226745,0.0,...,0.0,0.0,0.138342,0.280915,0.0,0.245328,0.0,0.0,0.385824,4.656012
2,0.079784,0.0,0.231710,0.0,0.0,0.186963,0.189660,0.186109,0.195873,0.0,...,0.0,0.0,0.133506,0.216937,0.0,0.227431,0.0,0.0,0.332119,4.942209
3,0.090358,0.0,0.275826,0.0,0.0,0.221281,0.209290,0.221356,0.239885,0.0,...,0.0,0.0,0.149898,0.277592,0.0,0.271456,0.0,0.0,0.420817,4.994409
4,0.066594,0.0,0.223903,0.0,0.0,0.183887,0.173088,0.179960,0.200711,0.0,...,0.0,0.0,0.125085,0.241765,0.0,0.219065,0.0,0.0,0.341056,4.000000


In [46]:
# try the function for another character
hello_2 = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, '2', s_path)

Block0103_2020_08_03_2.npy
1.0


In [37]:
hello_2.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,tassel_count
0,0.074282,0.0,0.272232,0.0,0.0,0.213224,0.195908,0.217338,0.228303,0.0,...,0.0,0.0,0.141018,0.291014,0.0,0.246560,0.0,0.0,0.392275,7.780586
1,0.081295,0.0,0.324027,0.0,0.0,0.244057,0.242304,0.243746,0.269518,0.0,...,0.0,0.0,0.165105,0.339280,0.0,0.294148,0.0,0.0,0.478314,7.321493
2,0.087211,0.0,0.246105,0.0,0.0,0.191530,0.193848,0.195882,0.208734,0.0,...,0.0,0.0,0.135215,0.234038,0.0,0.238300,0.0,0.0,0.356873,7.088444
3,0.088379,0.0,0.290204,0.0,0.0,0.225273,0.225619,0.229609,0.250859,0.0,...,0.0,0.0,0.149796,0.297610,0.0,0.274431,0.0,0.0,0.429273,7.997413
4,0.052577,0.0,0.250628,0.0,0.0,0.195973,0.188692,0.196736,0.217455,0.0,...,0.0,0.0,0.131372,0.280952,0.0,0.242157,0.0,0.0,0.385560,7.344028


In [39]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [40]:
%%time

# for single digit subwindows
catch_all_dfs = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, character, s_path)
    catch_all_dfs.append(features_df)

Block0103_2020_08_03_0.npy
1.0
Block0103_2020_08_03_1.npy
1.0
Block0103_2020_08_03_2.npy
1.0
Block0103_2020_08_03_3.npy
1.0
Block0103_2020_08_03_4.npy
1.0
Block0103_2020_08_03_5.npy
1.0
Block0103_2020_08_03_6.npy
1.0
Block0103_2020_08_03_7.npy
1.0
Block0103_2020_08_03_8.npy
1.0
Block0103_2020_08_03_9.npy
1.0
CPU times: user 11.9 s, sys: 1.21 s, total: 13.1 s
Wall time: 35.2 s


In [49]:
# get the predictions for double digits
character_list_double = list(range(10, 12))
character_list_double = list(map(str, character_list_double))

In [51]:
%%time

# for double digit subwindows
catch_all_dfs = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 3, 11, character, s_path)
    catch_all_dfs.append(features_df)

Block0103_2020_08_03_10.npy
1.0
Block0103_2020_08_03_11.npy
1.0
CPU times: user 2.23 s, sys: 221 ms, total: 2.45 s
Wall time: 5.69 s
